In [1]:
import pandas as pd 
import numpy as np
import talib as ta

In [2]:
# 读取并拼接trades数据
raw_data_02 = pd.read_csv('/Users/aming/project/python/课程代码合集/高频因子的思考和设计_2/data/BNBUSDT-trades-2025-04-02.zip')
raw_data_03 = pd.read_csv('/Users/aming/project/python/课程代码合集/高频因子的思考和设计_2/data/BNBUSDT-trades-2025-04-03.zip')
raw_data_04 = pd.read_csv('/Users/aming/project/python/课程代码合集/高频因子的思考和设计_2/data/BNBUSDT-trades-2025-04-04.zip')
raw_data = pd.concat([raw_data_02,raw_data_03,raw_data_04],axis=0)

In [3]:
raw_data

,id,price,qty,quote_qty,time,is_buyer_maker
0,1653714673,610.94,1.47,898.0818,1743552004762,False
1,1653714674,610.94,0.34,207.7196,1743552004787,False
2,1653714675,610.94,1.51,922.5194,1743552004793,False
3,1653714676,610.93,0.13,79.4209,1743552004879,True
4,1653714677,610.93,0.02,12.2186,1743552004879,True
...,...,...,...,...,...,...
1042790,1657705604,597.88,0.02,11.9576,1743811198826,True
1042791,1657705605,597.86,0.02,11.9572,1743811198826,True
1042792,1657705606,597.85,0.05,29.8925,1743811199160,False
1042793,1657705607,597.85,1.58,944.6030,1743811199160,False


In [4]:
raw_data['time'] = pd.to_datetime(raw_data['time'],unit='ms',utc=True)

In [5]:
raw_data

,id,price,qty,quote_qty,time,is_buyer_maker
0,1653714673,610.94,1.47,898.0818,2025-04-02 00:00:04.762000+00:00,False
1,1653714674,610.94,0.34,207.7196,2025-04-02 00:00:04.787000+00:00,False
2,1653714675,610.94,1.51,922.5194,2025-04-02 00:00:04.793000+00:00,False
3,1653714676,610.93,0.13,79.4209,2025-04-02 00:00:04.879000+00:00,True
4,1653714677,610.93,0.02,12.2186,2025-04-02 00:00:04.879000+00:00,True
...,...,...,...,...,...,...
1042790,1657705604,597.88,0.02,11.9576,2025-04-04 23:59:58.826000+00:00,True
1042791,1657705605,597.86,0.02,11.9572,2025-04-04 23:59:58.826000+00:00,True
1042792,1657705606,597.85,0.05,29.8925,2025-04-04 23:59:59.160000+00:00,False
1042793,1657705607,597.85,1.58,944.6030,2025-04-04 23:59:59.160000+00:00,False


In [6]:
# 滚动计算qty的分位数，确定大单
big_order = raw_data.qty.rolling(2000).quantile(0.95)

In [7]:
big_order

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
           ... 
1042790    2.33
1042791    2.33
1042792    2.33
1042793    2.33
1042794    2.33
Name: qty, Length: 3990427, dtype: float64

In [8]:
# 新增两列————大单主买量和大单主卖量
raw_data['big_order_buy'] = (raw_data['qty']>big_order)*(raw_data['qty'])*(~raw_data['is_buyer_maker'])
raw_data['big_order_sell'] = (raw_data['qty']>big_order)*(raw_data['qty'])*(raw_data['is_buyer_maker'])

In [9]:
# 降频处理
raw_data.index = raw_data['time']
raw_data = raw_data[['qty','big_order_buy','big_order_sell']]
resampled_raw_data = raw_data.resample('1min',label='right').agg({'qty':'sum',
                                             'big_order_buy':'sum',
                                             'big_order_sell':'sum'})

In [10]:
resampled_raw_data

,qty,big_order_buy,big_order_sell
time,,,
2025-04-02 00:01:00+00:00,395.95,0.00,0.00
2025-04-02 00:02:00+00:00,659.89,0.00,0.00
2025-04-02 00:03:00+00:00,730.97,0.00,0.00
2025-04-02 00:04:00+00:00,1183.54,169.20,250.94
2025-04-02 00:05:00+00:00,405.51,33.07,37.34
...,...,...,...
2025-04-04 23:56:00+00:00,168.98,72.06,34.40
2025-04-04 23:57:00+00:00,113.91,9.45,80.35
2025-04-04 23:58:00+00:00,42.39,2.33,18.76


In [11]:
# 读取k线数据，然后把k线和降频后的逐笔数据进行拼接
kline_02 = pd.read_csv('/Users/aming/project/python/课程代码合集/高频因子的思考和设计_2/data/BNBUSDT-1m-2025-04-02.zip')
kline_03 = pd.read_csv('/Users/aming/project/python/课程代码合集/高频因子的思考和设计_2/data/BNBUSDT-1m-2025-04-03.zip')
kline_04 = pd.read_csv('/Users/aming/project/python/课程代码合集/高频因子的思考和设计_2/data/BNBUSDT-1m-2025-04-04.zip')
kline = pd.concat([kline_02,kline_03,kline_04],axis=0)
kline

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1743552000000,610.94,611.37,610.93,611.01,395.95,1743552059999,241990.5704,514,275.45,168339.2811,0
1,1743552060000,611.01,611.34,611.00,611.03,659.89,1743552119999,403324.7321,750,301.92,184525.4573,0
2,1743552120000,611.04,611.38,611.02,611.30,730.97,1743552179999,446756.5093,699,577.38,352873.0463,0
3,1743552180000,611.30,611.31,611.08,611.08,1183.54,1743552239999,723457.8271,420,239.33,146281.3058,0
4,1743552240000,611.08,611.39,610.82,611.38,405.51,1743552299999,247784.4722,623,244.41,149354.0755,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1435,1743810900000,598.19,598.19,597.99,597.99,168.98,1743810959999,101075.4682,347,88.56,52973.8267,0
1436,1743810960000,598.00,598.00,597.88,597.93,113.91,1743811019999,68108.9761,188,21.28,12723.3140,0
1437,1743811020000,597.93,597.94,597.91,597.92,42.39,1743811079999,25345.8721,130,4.52,2702.6224,0
1438,1743811080000,597.91,597.92,597.80,597.81,45.65,1743811139999,27292.1141,188,1.32,789.1257,0


In [12]:
kline = kline[['close_time','open','high','low','close','volume']]
kline.index = pd.to_datetime(kline['close_time'],unit='ms',utc=True)
kline.index = kline.index.round('1min')
kline.drop('close_time',axis=1,inplace=True)

<ipython-input-12-073f66ceb941>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kline.drop('close_time',axis=1,inplace=True)


In [13]:
kline

,open,high,low,close,volume
close_time,,,,,
2025-04-02 00:01:00+00:00,610.94,611.37,610.93,611.01,395.95
2025-04-02 00:02:00+00:00,611.01,611.34,611.00,611.03,659.89
2025-04-02 00:03:00+00:00,611.04,611.38,611.02,611.30,730.97
2025-04-02 00:04:00+00:00,611.30,611.31,611.08,611.08,1183.54
2025-04-02 00:05:00+00:00,611.08,611.39,610.82,611.38,405.51
...,...,...,...,...,...
2025-04-04 23:56:00+00:00,598.19,598.19,597.99,597.99,168.98
2025-04-04 23:57:00+00:00,598.00,598.00,597.88,597.93,113.91
2025-04-04 23:58:00+00:00,597.93,597.94,597.91,597.92,42.39


In [14]:
# 把降频后的trades数据和kline数据拼接到一起
combined_z = pd.concat([kline,resampled_raw_data],axis=1)

In [15]:
combined_z
# 净大单主买/volume得到净主买占总成交量的占比

,open,high,low,close,volume,qty,big_order_buy,big_order_sell
2025-04-02 00:01:00+00:00,610.94,611.37,610.93,611.01,395.95,395.95,0.00,0.00
2025-04-02 00:02:00+00:00,611.01,611.34,611.00,611.03,659.89,659.89,0.00,0.00
2025-04-02 00:03:00+00:00,611.04,611.38,611.02,611.30,730.97,730.97,0.00,0.00
2025-04-02 00:04:00+00:00,611.30,611.31,611.08,611.08,1183.54,1183.54,169.20,250.94
2025-04-02 00:05:00+00:00,611.08,611.39,610.82,611.38,405.51,405.51,33.07,37.34
...,...,...,...,...,...,...,...,...
2025-04-04 23:56:00+00:00,598.19,598.19,597.99,597.99,168.98,168.98,72.06,34.40
2025-04-04 23:57:00+00:00,598.00,598.00,597.88,597.93,113.91,113.91,9.45,80.35
2025-04-04 23:58:00+00:00,597.93,597.94,597.91,597.92,42.39,42.39,2.33,18.76
2025-04-04 23:59:00+00:00,597.91,597.92,597.80,597.81,45.65,45.65,0.00,16.35


In [16]:
# 资金结构————成交量动量 1.pd.rolling() 2.talib 3.numpy
ma_20 = ta.MA(combined_z['close'],20)
diff_ = ta.MOM(ma_20) # 求差分
# 过去20个窗口价格是否单调递增？
if_increase = diff_.rolling(20).apply(lambda x: (x > 0).all())
# 统计单调递增下的净主买量
net_buy_v = combined_z.big_order_buy-combined_z.big_order_sell
net_buy_v_price_high = if_increase*net_buy_v
# 价格突破时，有多少净主买大单跟随？净主买大单占总成交量的比重？
# 价格是否一直稳定偏离vwap 